
# AWS EC2: Installing NVMe Driver, Configuring SCSI Persistent Reservations, and Expanding Storage

This notebook guides you through installing the AWS NVMe driver on an EC2 instance, configuring SCSI persistent reservations, and expanding storage for the C: drive on a Windows Server.

This process includes:
1. Backing up your instance
2. Installing the AWS NVMe driver
3. Configuring SCSI Persistent Reservations (optional)
4. Expanding storage for the C: drive by resizing the EBS volume

> **Note**: This notebook is intended for users with a basic understanding of PowerShell and Windows Server on AWS.



## Step 1: Backup the Instance

Creating an AMI backup ensures you can revert any changes safely.

1. In the AWS Management Console, navigate to **EC2 > Instances**.
2. Stop the instance by selecting **Instance State > Stop Instance**.
3. After the instance is stopped, go to **Actions > Image and templates > Create image** and create a backup.
4. Once the backup is created, start the instance by selecting **Instance State > Start Instance**.



## Step 2: Connect to Your Instance

1. Open **Remote Desktop Connection** on your computer and connect to your EC2 instance.
2. Log in as the local administrator to proceed with the installation.




## Step 3: Download the Latest AWS NVMe Driver

Choose one of the following methods to download and extract the NVMe driver.

### Option 1: Download Using a Browser
1. Download the latest driver package from [AWS NVMe driver download link](https://s3.amazonaws.com/ec2-windows-drivers-downloads/NVMe/Latest/AWSNVMe.zip).
2. Extract the zip archive to a folder.

### Option 2: Download Using PowerShell
Execute the following PowerShell commands to download and extract the driver:

```powershell
Invoke-WebRequest https://s3.amazonaws.com/ec2-windows-drivers-downloads/NVMe/Latest/AWSNVMe.zip -outfile $env:USERPROFILE\nvme_driver.zip
Expand-Archive $env:USERPROFILE\nvme_driver.zip -DestinationPath $env:USERPROFILE\nvme_driver
```

> **Note**: If you encounter TLS errors on Windows Server 2016 or earlier, enable TLS 1.2 for your session by running:

```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
```



## Step 4: Install the AWS NVMe Driver

1. Open PowerShell as an administrator.
2. Navigate to the directory where you extracted the NVMe driver.
3. Run the installation script:

```powershell
cd $env:USERPROFILE\nvme_driver
.\install.ps1
```

### Optional: Enable SCSI Persistent Reservations
For Windows Server 2016 or later, enable SCSI persistent reservations if needed:

```powershell
.\install.ps1 -EnableSCSIPersistentReservations $true
```

Disable SCSI persistent reservations if not needed:

```powershell
.\install.ps1 -EnableSCSIPersistentReservations $false
```

> **Note**: SCSI persistent reservations are only necessary if you are using Windows Server Failover Clustering with shared Amazon EBS storage.



## Step 5: Configure SCSI Persistent Reservations (Registry Option)

After installing AWS NVMe driver version 1.5.0 or later, you can configure SCSI persistent reservations in the registry.

1. Open PowerShell as an administrator.
2. Set the registry path variable:

```powershell
$registryPath = "HKLM:\SYSTEM\CurrentControlSet\Services\AWSNVMe\Parameters\Device"
```

3. **Enable** SCSI persistent reservations:

```powershell
Set-ItemProperty -Path $registryPath -Name EnableSCSIPersistentReservations -Value 1
```

4. **Disable** SCSI persistent reservations:

```powershell
Set-ItemProperty -Path $registryPath -Name EnableSCSIPersistentReservations -Value 0
```

5. Restart the instance to apply changes:

```powershell
Restart-Computer
```

> **Important**: Only enable SCSI persistent reservations if you need shared Amazon EBS storage with Windows Server Failover Clustering.



## Step 6: Expanding the EBS Volume for the C: Drive

To increase your root volume size:

1. In the **EC2 Console**, go to **Elastic Block Store > Volumes**.
2. Select your root EBS volume (usually labeled `/dev/sda1`).
3. Choose **Modify Volume** and set a larger size.
4. After modifying the volume, go back to your EC2 instance and open **Disk Management**.
5. Locate the **C:** drive, right-click, and select **Extend Volume** to use the additional space.

Your C: drive should now have the expanded space from the resized EBS volume.



## Conclusion

You have now successfully:
1. Created a backup of your EC2 instance.
2. Downloaded and installed the AWS NVMe driver.
3. Configured (or disabled) SCSI persistent reservations as required.
4. Expanded the storage on your C: drive by resizing the root EBS volume.

These steps help ensure that your SSD storage is configured for optimal use in your EC2 environment.
